In [50]:
import torch
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
import torch.nn as nn

from transformers import ConvNextV2ForImageClassification

class LayerNorm2d(nn.LayerNorm):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.permute(0, 2, 3, 1)
        x = torch.nn.functional.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x
n_inputs = 768
hidden_size = 256 # default is 2048
n_outputs = 10

#model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT)
model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-pico-1k-224")
#model = nn.Sequential(*list(model.children())[:-1])
#num_features = model[-1][-1].conv3.out_channels
sequential_layers = nn.Sequential(
    LayerNorm2d((n_inputs,), eps=1e-06, elementwise_affine=True),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(n_inputs, hidden_size, bias=True),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(hidden_size, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, n_outputs),
    nn.LogSoftmax(dim=1)
)
model.classifier = sequential_layers
'''
layers = [(name, param.requires_grad) for name, param in model.named_parameters()]
for i in range(len(layers)):
    print(layers[i])
'''
'''
n_inputs = None
for name, child in model.named_children():
    if name == 'classifier':
        for sub_name, sub_child in child.named_children():
            if sub_name == '2':
                n_inputs = sub_child.in_features
'''
num = 0
for name, param in model.named_parameters():
    if (param.requires_grad):
        num += 1
    #if (num > 159): # for freezing layers 1, 2, 3, and 4
    #if (num > 129): # for freezing layers 1, 2, and 3
    #if (num > 72): # for freezing layers 1 and 2
    #if (num > 33): #for freezing layer 1
    #if (num >= 0): # for no freezing
        #param.requires_grad = True
    #else:
        #param.requires_grad = False
print(num)

150


In [88]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model = nn.Sequential(*list(model.children())[:-4])
num_features = model[-1][-1].conv3.out_channels
#model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-atto-1k-224")
#num_features = 320
#num_features = 128
dropout = 0.3
model.add_module('avgpool', nn.AdaptiveAvgPool2d(1))
model.add_module('flatten', nn.Flatten())
model.add_module('fc1', nn.Linear(num_features, 256))
model.add_module('gelu', nn.GELU())
model.add_module('dropout5', nn.Dropout(dropout))
model.add_module('fc2', nn.Linear(256, 10))
model.add_module('softmax', nn.Softmax(dim=1))

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

model.to(DEVICE)
x_sample    = torch.rand(6, 3, 256, 256)
torchsummaryX.summary(model, x_sample.to(DEVICE))

Device:  cuda
                                     Kernel Shape       Output Shape  \
Layer                                                                  
0_0                                 [3, 64, 7, 7]  [6, 64, 128, 128]   
1_1                                          [64]  [6, 64, 128, 128]   
2_2                                             -  [6, 64, 128, 128]   
3_3                                             -    [6, 64, 64, 64]   
4_4.0.Conv2d_conv1                 [64, 64, 1, 1]    [6, 64, 64, 64]   
5_4.0.BatchNorm2d_bn1                        [64]    [6, 64, 64, 64]   
6_4.0.ReLU_relu                                 -    [6, 64, 64, 64]   
7_4.0.Conv2d_conv2                 [64, 64, 3, 3]    [6, 64, 64, 64]   
8_4.0.BatchNorm2d_bn2                        [64]    [6, 64, 64, 64]   
9_4.0.ReLU_relu                                 -    [6, 64, 64, 64]   
10_4.0.Conv2d_conv3               [64, 256, 1, 1]   [6, 256, 64, 64]   
11_4.0.BatchNorm2d_bn3                      [256] 

/home/maxwell/.local/lib/python3.7/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_0,"[3, 64, 7, 7]","[6, 64, 128, 128]",9408.0,154140672.0
1_1,[64],"[6, 64, 128, 128]",128.0,64.0
2_2,-,"[6, 64, 128, 128]",NaN,NaN
3_3,-,"[6, 64, 64, 64]",NaN,NaN
4_4.0.Conv2d_conv1,"[64, 64, 1, 1]","[6, 64, 64, 64]",4096.0,16777216.0
...,...,...,...,...
73_fc1,"[512, 256]","[6, 256]",131328.0,131072.0
74_gelu,-,"[6, 256]",NaN,NaN
75_dropout5,-,"[6, 256]",NaN,NaN


In [9]:
#from transformers import AutoImageProcessor, ConvNextV2Model
#model = resnet50(weights=ResNet50_Weights.DEFAULT)
#model = nn.Sequential(*list(model.children())[:-3])
#from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
#from torchvision.models import convnext_small, ConvNeXt_Small_Weights
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
#from torchvision.models import convnext_large, ConvNeXt_Large_Weights
import torchsummaryX
from functools import partial
import torch
from torch import nn, Tensor
from torch.nn import functional as F

class LayerNorm2d(nn.LayerNorm):
    def forward(self, x: Tensor) -> Tensor:
        x = x.permute(0, 2, 3, 1)
        x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x

#model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT)
#model = convnext_small(weights=ConvNeXt_Small_Weights.DEFAULT)
model = convnext_base(weights=ConvNeXt_Base_Weights.DEFAULT)
#model = convnext_large(weights=ConvNeXt_Large_Weights.DEFAULT)
model.features = model.features[:-4]
#model.avgpool = nn.AdaptiveAvgPool2d(1)
norm_layer = partial(LayerNorm2d, eps=1e-6)
lastconv_output_channels = 256
num_classes = 10
model.classifier = nn.Sequential(
    norm_layer(lastconv_output_channels), nn.Flatten(1), nn.Linear(lastconv_output_channels, num_classes)
)
#image_processor = AutoImageProcessor.from_pretrained("facebook/convnextv2-atto-1k-224")
#model = ConvNextV2Model.from_pretrained("facebook/convnextv2-atto-1k-224")
#model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.DEFAULT)
#model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-nano-1k-224")



DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

model.to(DEVICE)
x_sample    = torch.rand(6, 3, 256, 256)
torchsummaryX.summary(model, x_sample.to(DEVICE))
'''
print(len(model.features))
print(model.features[-1])
'''

Device:  cuda
                                                      Kernel Shape  \
Layer                                                                
0_features.0.Conv2d_0                               [3, 128, 4, 4]   
1_features.0.LayerNorm2d_1                                   [128]   
2_features.1.0.block.Conv2d_0                       [1, 128, 7, 7]   
3_features.1.0.block.Permute_1                                   -   
4_features.1.0.block.LayerNorm_2                             [128]   
5_features.1.0.block.Linear_3                           [128, 512]   
6_features.1.0.block.GELU_4                                      -   
7_features.1.0.block.Linear_5                           [512, 128]   
8_features.1.0.block.Permute_6                                   -   
9_features.1.0.StochasticDepth_stochastic_depth                  -   
10_features.1.1.block.Conv2d_0                      [1, 128, 7, 7]   
11_features.1.1.block.Permute_1                                  -   
12_fea

/home/maxwell/.local/lib/python3.7/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


'\nprint(len(model.features))\nprint(model.features[-1])\n'